In [1]:
#Import related libraries
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import torch as th
import torch.nn as nn
import dgl
from dgl import function as fn
import pandas as pd
import numpy as np
import networkx as nx
import torch.nn.functional as F
from torch.autograd import Variable
import time

In [2]:
class TAGConv(nn.Module):
    def __init__(self,
                 in_feats,    
                 out_feats,   
                 k=2,         
                 bias=True,   
                 activation=None): 

        super(TAGConv, self).__init__()   
      
        self._in_feats = in_feats  
        self._out_feats = out_feats 
        self._k = k
        self._activation = activation

        self.lin = nn.Linear(in_feats * (self._k + 1), out_feats, bias=bias)  
        #self.dropout = nn.Dropout(0.5)
        

        self.reset_parameters() 


    def reset_parameters(self):      
        """Reinitialize learnable parameters."""
        gain = nn.init.calculate_gain('relu')     
        nn.init.xavier_normal_(self.lin.weight, gain=gain)   

        
    def forward(self, graph, feat):    
        graph = graph.local_var()           
        
        norm = th.pow(graph.in_degrees().float().clamp(min=1), -0.5) 
    
        
        shp = norm.shape + (1,) * (feat.dim() - 1)    
        
        norm = th.reshape(norm, shp).to(feat.device)  

        #D-1/2 A D -1/2 X
        fstack = [feat]      
        for _ in range(self._k):

            rst = fstack[-1] * norm    
            
            graph.ndata['h'] = rst     

            graph.update_all(fn.copy_src(src='h', out='m'),   
                             fn.sum(msg='m', out='h'))     
           
            
            rst = graph.ndata['h']     
            rst = rst * norm           
            fstack.append(rst)         

        rst = self.lin(th.cat(fstack, dim=-1))    

        if self._activation is not None:
            rst = self._activation(rst)         

        return rst
    

In [3]:
class GCN(nn.Module):
   
    def __init__(self,input_dim,hidden_size,num_classes):
        super(GCN, self).__init__()
        
       
        self.GRU = nn.GRU(input_size=input_dim, hidden_size=hidden_size, num_layers=1)
        
       
        self.gcn1 = TAGConv(12,hidden_size,k=1)

        
        self.gcn2 = TAGConv(hidden_size,num_classes,k=1)
  

    def forward(self, graph, feature):
       
        h = self.GRU(feature)

        h = F.relu(self.gcn1(graph,h[0])) #Hidden layer operation, absorbing information from neighboring nodes in the first layer

        h = self.gcn2(graph, h)           #Fully connected layer operation, absorbing information from neighboring nodes in the second layer
        return h
        

In [4]:
#Input adjacency matrix
A_city=pd.read_excel(r'D:\0_Shay\Practise\jupyter\CSJ\LJJZ.xlsx',index_col=[0])
A_city.fillna(value=0,inplace=True)

In [5]:
for i in A_city.columns:
    for j in A_city.index:
        if i==j:
            A_city[i][j]=1
A_city_array=np.array(A_city)
G1 = dgl.DGLGraph()

for i in range(len(A_city_array)):
    for j in range(len(A_city_array)):
        if A_city_array[i][j]==1:
            G1.add_edge(i, j)
        else:
            pass
G1

D:\anaconda\anaconda\envs\torch\lib\site-packages\dgl\base.py:45: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  return warnings.warn(message, category=category, stacklevel=1)
D:\anaconda\anaconda\envs\torch\lib\site-packages\dgl\base.py:45: DGLWarning: DGLGraph.add_edge is deprecated. Please use DGLGraph.add_edges
  return warnings.warn(message, category=category, stacklevel=1)


Graph(num_nodes=307, num_edges=1969,
      ndata_schemes={}
      edata_schemes={})

In [6]:
#Input feature matrix
df = pd.read_excel(r'D:\0_Shay\Practise\jupyter\CSJ\GYH62.xlsx',index_col=[0])

In [7]:
def gen_lab(dataset, start, end, experience, future):   
    data = []
    labels = []
    data_list = []
    labels_list = []

    real_start = start + experience     

    for i in range(real_start, end):    
        data.append(dataset.iloc[i-experience:i])    
        labels.append(dataset.iloc[i:i+future])    #i:i+future
    
    for j in range(len(data)):
        data_tensor = th.Tensor(np.array(data[j].T))
        data_list.append(data_tensor)
    
    for k in range(len(labels)):
        labels_tensor = th.Tensor(np.array(labels[k].T))
        labels_list.append(labels_tensor)
    
    return th.stack(data_list), th.stack(labels_list)

In [8]:
#Divide the dataset according to 6:2:2
train_x,train_y = gen_lab(df,0,30,6,1)
valid_x,valid_y = gen_lab(df,30,40,6,1)
test_x,test_y = gen_lab(df,40,50,6,1)
train_x.shape,train_y.shape,valid_x.shape,test_x.shape,valid_y.shape,test_y.shape

(torch.Size([24, 307, 6]),
 torch.Size([24, 307, 1]),
 torch.Size([4, 307, 6]),
 torch.Size([4, 307, 6]),
 torch.Size([4, 307, 1]),
 torch.Size([4, 307, 1]))

In [9]:
def gcn_trainer(network,graph,input_data,label_data,training_times,
                optimizer,criterion,loss_list,dur_list):
   
    #loss_list = loss_list
    #network = network

    for epoch in range(training_times):
        t0 = time.time()
        network.train()
        out = network(graph,input_data)          
        
        #criterion = criterion
        loss = criterion(out,label_data)

        
        #optimizer = optimizer
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_list.append(loss)
        dur_list.append(time.time() - t0)
        

        
        if (epoch+1) % 70 == 0: #epoch=70
            #acc = evaluate(net, g, features, labels, test_mask)
            print("Epoch {:04d} | MAE_Test_Loss {:.4f}".format(epoch+1, loss.item())) 
            
            

In [10]:
mymodel=GCN(6,12,1);GPGCN_Loss_list=[];GPGCN_Loss_list.clear();GPGCN_MAE_Dur_list=[]
for i in range(len(train_x)):
    print('Batch{:d}: '.format(i+1),end='')
    gcn_trainer(mymodel,G1,train_x[i],train_y[i],70,th.optim.Adam(mymodel.parameters(),lr=1e-4),nn.L1Loss(),GPGCN_Loss_list,GPGCN_MAE_Dur_list)


Batch1: 

D:\anaconda\anaconda\envs\torch\lib\site-packages\torch\autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


Epoch 0070 | MAE_Test_Loss 0.0254
Batch2: Epoch 0070 | MAE_Test_Loss 0.0151
Batch3: Epoch 0070 | MAE_Test_Loss 0.0056
Batch4: Epoch 0070 | MAE_Test_Loss 0.0056
Batch5: Epoch 0070 | MAE_Test_Loss 0.0055
Batch6: Epoch 0070 | MAE_Test_Loss 0.0053
Batch7: Epoch 0070 | MAE_Test_Loss 0.0053
Batch8: Epoch 0070 | MAE_Test_Loss 0.0058
Batch9: Epoch 0070 | MAE_Test_Loss 0.0159
Batch10: Epoch 0070 | MAE_Test_Loss 0.0083
Batch11: Epoch 0070 | MAE_Test_Loss 0.0078
Batch12: Epoch 0070 | MAE_Test_Loss 0.0093
Batch13: Epoch 0070 | MAE_Test_Loss 0.0099
Batch14: Epoch 0070 | MAE_Test_Loss 0.0092
Batch15: Epoch 0070 | MAE_Test_Loss 0.0064
Batch16: Epoch 0070 | MAE_Test_Loss 0.0074
Batch17: Epoch 0070 | MAE_Test_Loss 0.0069
Batch18: Epoch 0070 | MAE_Test_Loss 0.0064
Batch19: Epoch 0070 | MAE_Test_Loss 0.0063
Batch20: Epoch 0070 | MAE_Test_Loss 0.0068
Batch21: Epoch 0070 | MAE_Test_Loss 0.0129
Batch22: Epoch 0070 | MAE_Test_Loss 0.0084
Batch23: Epoch 0070 | MAE_Test_Loss 0.0079
Batch24: Epoch 0070 | MAE_Te

In [11]:
def gcn_valid(network,test_input,test_label,criterion,loss_list,out_list):

    for i in range(len(test_input)):
        loss = criterion(network(G1,test_input[i]),test_label[i])
        loss_list.append(loss)
        out = network(G1,test_input[i])
        out_list.append(out)

In [12]:
GPGCN_valid_loss_list=[];GPGCN_valid_loss_list.clear();GPGCN_valid_out_list=[];GPGCN_valid_out_list.clear();
gcn_valid(mymodel,valid_x,valid_y,nn.L1Loss(),GPGCN_valid_loss_list,GPGCN_valid_out_list)

In [13]:
def gcn_tester(network,test_input,test_label,criterion,loss_list,out_list):
    for i in range(len(test_input)):
        loss = criterion(network(G1,test_input[i]),test_label[i])
        loss_list.append(loss)
        out = network(G1,test_input[i])
        out_list.append(out)

In [14]:
GPGCN_test_loss_list=[];GPGCN_test_loss_list.clear();GPGCN_test_out_list=[];GPGCN_test_out_list.clear()
gcn_tester(mymodel,test_x,test_y,nn.L1Loss(),GPGCN_test_loss_list,GPGCN_test_out_list)

In [15]:
#The trained Yangtze River Delta model in the paper
CSJmodel = th.load(r'D:\0_Shay\Practise\jupyter\trained\aftertained\CSJ.pth')

In [16]:
GPGCN_test_loss_list=[];GPGCN_test_loss_list.clear();GPGCN_test_out_list=[];GPGCN_test_out_list.clear()
gcn_tester(CSJmodel,test_x,test_y,nn.L1Loss(),GPGCN_test_loss_list,GPGCN_test_out_list)

In [17]:
#The following is a multi-step prediction under recursive strategy
def gen_labpre(dataset, start, end, experience, future):   
    data = []
    labels = []
    data_list = []
    labels_list = []

    real_start = start + experience     

    for i in range(real_start, end+1):    
        data.append(dataset.iloc[i-experience:i])    
        labels.append(dataset.iloc[i:i+future])    #i:i+future
    
    for j in range(len(data)):
        data_tensor = th.Tensor(np.array(data[j].T))
        data_list.append(data_tensor)
    
    for k in range(len(labels)):
        labels_tensor = th.Tensor(np.array(labels[k].T))
        labels_list.append(labels_tensor)
    
    return th.stack(data_list), th.stack(labels_list)

testlist = pd.read_excel(r'D:\0_Shay\Practise\jupyter\trained\prediction4.xlsx',index_col=[0])

for i in range(4):
    test_xpre,test_ypre = gen_labpre(testlist,0,6+i,6,0)
    predict1=CSJmodel(G1,test_xpre[-1])
    predict1
    out_list = predict1.tolist()
    out_list = np.transpose(out_list).tolist()
    testlist=testlist.append(out_list)